In [1]:
import numpy as np
import requests

In this homework, you will implement the Huffmann compression algorithm discussed in class.

We are going compress Shakespear's Hamlet, so let us download the text (courtesy of Project Gutenberg). Borges' [The Library of Babel](https://en.wikipedia.org/wiki/The_Library_of_Babel) would be even more fitting to compress. Unfortunately it is not yet in the public domain...

In [2]:
URL = "https://github.com/amsqi/iit21-homework/raw/main/material/hamlet.txt"
hamlet = requests.get(URL).content.decode("ascii", errors="ignore")
hamlet = hamlet
len(hamlet)

179096

Thus we have around 180 KB to compress!

# 1. Huffman Compression

Throughout this exercise, we use the following conventions:
- **probability distributions** are represented by dictionaries that map symbols to probabilities
- **bitstrings** are represented by Python lists or tuples that contain 0s and 1s (for simplicity)
- **codes** are represented by dictionaries that map symbols to bitstrings

The following function can be used to compute the average length of a code $C$ under a probability distribution $P$:

In [3]:
def L(C, P):
    return sum(P[x] * len(C[x]) for x in P if P[x] > 0)

And the following function tests if a given code is a prefix code:

In [4]:
def is_prefix_code(C):
    # sort codewords by length
    codewords = sorted(C.values(), key=len)

    # check if any codeword is prefix of any other
    for i, first in enumerate(codewords):
        l = len(first)
        for second in codewords[i + 1 :]:
            if second[:l] == first:
                return False
    return True

Here is a simple example:

In [5]:
P = {"X": 0.5, "O": 0.3, "L": 0.2}

C = {"X": [0], "O": [1, 0], "L": [1, 1]}
assert is_prefix_code(C) and L(C, P) == 1.5

C = {"X": [0], "O": [0, 0], "L": [0, 0, 0]}
assert not is_prefix_code(C) and np.round(L(C, P), 1) == 1.7

**Your first task is to implement Huffmann's algorithm for creating the optimal prefix code for a given probability distribution.**

Your function should take as input a probability distribution $P$ and return as output a code $C$ (in the format described above).

In [ ]:
def huffman_code(P):
    # TODO: can you implement this?
    return ...

Here are some test cases that you can run to make sure that your code works:

In [ ]:
# the probability distribution from class
P = {"A": 0.25, "B": 0.25, "C": 0.2, "D": 0.15, "E": 0.15}
C = huffman_code(P)
assert is_prefix_code(C)
assert L(C, P) == 2.3

# uniform probability distribution over a byte (8 bits)
P = {k: 1 / 256 for k in range(256)}
C = huffman_code(P)
assert is_prefix_code(C)
assert L(C, P) == 8

# letter distribution of the English language (from MacKay's book)
P = {
    "a": 0.0575,
    "b": 0.0128,
    "c": 0.0263,
    "d": 0.0285,
    "e": 0.0913,
    "f": 0.0173,
    "g": 0.0133,
    "h": 0.0313,
    "i": 0.0599,
    "j": 0.0006,
    "k": 0.0084,
    "l": 0.0335,
    "m": 0.0235,
    "n": 0.0596,
    "o": 0.0689,
    "p": 0.0192,
    "q": 0.0008,
    "r": 0.0508,
    "s": 0.0567,
    "t": 0.0706,
    "u": 0.0334,
    "v": 0.0069,
    "w": 0.0119,
    "x": 0.0073,
    "y": 0.0164,
    "z": 0.0007,
    " ": 0.1928,
}
C = huffman_code(P)
assert is_prefix_code(C)
assert np.round(L(C, P), 2) == 4.15

When compressing strings in practice, what probability distribution $P$ should we use to construct the Huffman code? A simple guess is the empirical probability distribution of the string. It is given by
$$P(x) = \frac {N_n} N,$$
where $N$ is the length of the string and $N_x$ the number of times that symbol $x$ appears in it.

**Write a function that takes as input a string or list of symbols and returns as output its empirical probability distribution.**

In [ ]:
def empirical(s):
    # TODO: can you implement this?
    return ...

In [ ]:
assert empirical("ABBBBA") == {"A": 2 / 6, "B": 4 / 6}

We are now in a good situation to compress an arbitrary unknown string.

**Write a function that takes as input a string and as output returns a pair `(C,bs)`, where `C` is a Huffman code for the empirical probability distribution and where `bs` is a bitstring containing the compression of the input.**

In [ ]:
def huffman_compress(s):
    # TODO: can you implement this?
    C = ...
    bs = ...
    return C, bs

For your convenience, we already programmed a decompressor:

In [ ]:
from collections import defaultdict


def huffman_decompress(C, bs):
    def tree():
        return defaultdict(tree)

    # build a binary tree for "fast" decompression
    root = tree()
    for x, cw in C.items():
        node = root
        for b in cw[:-1]:
            node = node[b]
        node[cw[-1]] = x

    # walk tree bit by bit
    result = ""
    node = root
    for b in bs:
        node = node[b]

        # if we are at a leave, emit the corresponding symbol and return to root
        if not isinstance(node, dict):
            result += node
            node = root
    return result

Here is a simple test to make sure that your compressor works fine:

In [ ]:
PLAIN_TEXT = "Welcome to the quantum quest."
C, bs = huffman_compress(PLAIN_TEXT)
assert huffman_decompress(C, bs) == PLAIN_TEXT
print(f"Successfully compressed {len(PLAIN_TEXT)*8} bits into {len(bs)} bits")

Of course, most savings here come from the fact that we don't use the full range of characters. What compression rate do you achieve when compressing Hamlet? Run the following code to find out:

In [ ]:
# compress hamlet
C, compressed = huffman_compress(hamlet)

# make sure it decompresses correctly
assert huffman_decompress(C, compressed) == hamlet

# print compression ratio
compressed_bytes = len(compressed) / 8
R = compressed_bytes / len(hamlet)
print(f"Compression rate: {R:2.0%}")

**Bonus challenges for the Huffman problem (completely optional):**

1. Download the `enwik8` data set from http://mattmahoney.net/dc/textdata.html. It contains the first 100 MB of a Wikipedia data dump. Optimize your Huffman compressor so that it can compress this data set in a few seconds! Does the decompressor need any optimizing, too?
2. In a realistic compressor, you would return the codebook `C` as part of the bitstring `bs`. Implement this and see to which extent this impacts your compression rate.